In [1]:
# Import required libraries
import pandas as pd
import math
from google.colab import files
import io

# Constants
C_p_default = 4181  # J/kg*K for water
h_heat_default = 0.75  # heating efficiency
h_stir_default = 0.9  # stirring efficiency
Np_default = 0.79  # power number for impeller
filtration_lower_limit = 1  # kWh/ton
filtration_upper_limit = 10  # kWh/ton

In [2]:
def celsius_to_kelvin(temp_celsius):
    return temp_celsius + 273.15

def calculate_heating_energy(m_mix, T_r_celsius, T_0_celsius, reaction_time, rate_heat_loss_perkelvin, C_p=C_p_default, h_heat=h_heat_default):
    # Add debug prints
    print(f"\nHeating energy calculation debug:")
    print(f"m_mix: {m_mix}")
    print(f"T_r_celsius: {T_r_celsius}")
    print(f"T_0_celsius: {T_0_celsius}")
    print(f"reaction_time: {reaction_time}")
    print(f"rate_heat_loss_perkelvin: {rate_heat_loss_perkelvin}")
    print(f"C_p: {C_p}")
    print(f"h_heat: {h_heat}")

    T_r = celsius_to_kelvin(T_r_celsius)
    T_0 = celsius_to_kelvin(T_0_celsius)

    print(f"T_r (K): {T_r}")
    print(f"T_0 (K): {T_0}")

    Q_heat = C_p * m_mix * (T_r - T_0)
    Q_loss = rate_heat_loss_perkelvin * (T_r - T_0) * reaction_time
    Q_react = (Q_heat + Q_loss) / h_heat

    print(f"Q_heat: {Q_heat}")
    print(f"Q_loss: {Q_loss}")
    print(f"Q_react: {Q_react}")

    return Q_react

def calculate_stirring_energy(rho_mix, N, d, reaction_time, Np=Np_default, h_stir=h_stir_default):
    E_stir = (Np * rho_mix * (N**3) * (d**5) * reaction_time) / h_stir
    return E_stir

def calculate_drying_energy(Cp_liq, m_liq, T_boil_celsius, T_0_celsius, H_vap, m_vap, eta_dry):
    print(f"\nDrying energy calculation debug:")
    print(f"Cp_liq: {Cp_liq}")
    print(f"m_liq: {m_liq}")
    print(f"T_boil_celsius: {T_boil_celsius}")
    print(f"T_0_celsius: {T_0_celsius}")
    print(f"H_vap: {H_vap}")
    print(f"m_vap: {m_vap}")
    print(f"eta_dry: {eta_dry}")

    T_boil = celsius_to_kelvin(T_boil_celsius)
    T_0 = celsius_to_kelvin(T_0_celsius)

    print(f"T_boil (K): {T_boil}")
    print(f"T_0 (K): {T_0}")

    Q_heat = Cp_liq * m_liq * (T_boil - T_0)
    Q_vap = H_vap * m_vap
    Q_dry = (Q_heat + Q_vap) / eta_dry

    print(f"Q_heat: {Q_heat}")
    print(f"Q_vap: {Q_vap}")
    print(f"Q_dry: {Q_dry}")

    return Q_dry

def calculate_filtration_energy(dry_material_mass_kg, use_upper_limit=False):
    print(f"\nFiltration energy calculation debug:")
    print(f"dry_material_mass_kg: {dry_material_mass_kg}")
    print(f"use_upper_limit: {use_upper_limit}")

    dry_material_mass_ton = dry_material_mass_kg / 1000
    print(f"dry_material_mass_ton: {dry_material_mass_ton}")

    if use_upper_limit:
        energy = filtration_upper_limit * dry_material_mass_ton
        print(f"Using upper limit: {filtration_upper_limit} kWh/ton")
        print(f"Calculated energy: {energy} kWh")
    else:
        energy = filtration_lower_limit * dry_material_mass_ton
        print(f"Using lower limit: {filtration_lower_limit} kWh/ton")
        print(f"Calculated energy: {energy} kWh")

    return energy

In [3]:
# Upload your Excel file
print("Please upload your Excel file:")
uploaded = files.upload()

# Get the filename from the uploaded files
filename = list(uploaded.keys())[0]

# Read the Excel file, skipping the first two rows and using row 2 as headers
df = pd.read_excel(filename, sheet_name='Batch Data', skiprows=2)

# Store original column names before cleaning
original_column_names = df.columns.tolist()

# Clean up column names for calculations
df.columns = [col.split('\n')[1].strip('()') if '\n' in str(col) else col for col in df.columns]

# Fill default values for optional parameters
df['C_p'] = df['C_p'].fillna(C_p_default)
df['h_heat'] = df['h_heat'].fillna(h_heat_default)
df['Np'] = df['Np'].fillna(Np_default)
df['h_stir'] = df['h_stir'].fillna(h_stir_default)
df['Cp_liq'] = df['Cp_liq'].fillna(4184)
df['T_boil'] = df['T_boil'].fillna(100)
df['H_vap'] = df['H_vap'].fillna(2260000)
df['eta_dry'] = df['eta_dry'].fillna(0.8)

# Calculate reaction time in seconds
df['reaction_time'] = df['reaction_time_hours'] * 3600

# Initialize energy columns
df['heating_energy_J'] = 0.0
df['stirring_energy_J'] = 0.0
df['filtration_energy_kWh'] = 0.0
df['drying_energy_J'] = 0.0
df['total_energy_kJ'] = 0.0
df['total_energy_kWh'] = 0.0
df['total_energy_MJ'] = 0.0

# Display current data
print("\nData after preprocessing:")
print(df.head())

Please upload your Excel file:


Saving input_bioprocess_paccino_data_sheet.xlsx to input_bioprocess_paccino_data_sheet.xlsx

Data after preprocessing:
  batch_description  date  m_mix  T_r  T_0  reaction_time_hours  \
0             100 L   NaN    100   30   25                   48   
1             500 L   NaN    500   30   25                   48   
2            1000 L   NaN   1000   30   25                   48   
3            5000 L   NaN   5000   30   25                   48   
4           10000 L   NaN  10000   30   25                   48   

   rate_heat_loss_perkelvin     C_p  h_heat  rho_mix  ...      H_vap  eta_dry  \
0                     0.712  4181.0    0.75     1000  ...  2260000.0      0.8   
1                     2.081  4181.0    0.75     1000  ...  2260000.0      0.8   
2                     3.303  4181.0    0.75     1000  ...  2260000.0      0.8   
3                     9.659  4181.0    0.75     1000  ...  2260000.0      0.8   
4                    15.333  4181.0    0.75     1000  ...  2260000.0     

In [4]:
# Process each row (batch)
for index, row in df.iterrows():
    try:
        print(f"\nProcessing batch: {row['batch_description']}")
        print(f"Input values:")
        print(f"m_mix: {row['m_mix']}")
        print(f"T_r: {row['T_r']}")
        print(f"T_0: {row['T_0']}")
        print(f"reaction_time: {row['reaction_time']}")
        print(f"rate_heat_loss_perkelvin: {row['rate_heat_loss_perkelvin']}")
        print(f"C_p: {row['C_p']}")
        print(f"h_heat: {row['h_heat']}")

        # Calculate heating energy
        Q_react = calculate_heating_energy(
            row['m_mix'],
            row['T_r'],
            row['T_0'],
            row['reaction_time'],
            row['rate_heat_loss_perkelvin'],
            row['C_p'],
            row['h_heat']
        )
        df.at[index, 'heating_energy_J'] = Q_react

        # Calculate stirring energy
        E_stir = calculate_stirring_energy(
            row['rho_mix'],
            row['N'],
            row['d'],
            row['reaction_time'],
            row['Np'],
            row['h_stir']
        )
        df.at[index, 'stirring_energy_J'] = E_stir

        # Calculate filtration energy if dry material mass is provided
        if pd.notna(row['dry_material_mass_kg']):
            filtration_energy = calculate_filtration_energy(
                row['dry_material_mass_kg'],
                row.get('use_upper_limit_filtration', False)
            )
            df.at[index, 'filtration_energy_kWh'] = filtration_energy

        # Calculate drying energy if required parameters are provided
        if pd.notna(row['m_liq']) and pd.notna(row['m_vap']):
            Q_dry = calculate_drying_energy(
                row['Cp_liq'],
                row['m_liq'],
                row.get('T_boil', 100),
                row['T_0'],
                row['H_vap'],
                row['m_vap'],
                row['eta_dry']
            )
            df.at[index, 'drying_energy_J'] = Q_dry

        # Calculate total energy in different units
        total_energy_J = Q_react + E_stir + (df.at[index, 'filtration_energy_kWh'] * 3600000) + df.at[index, 'drying_energy_J']
        df.at[index, 'total_energy_kJ'] = total_energy_J / 1000
        df.at[index, 'total_energy_kWh'] = total_energy_J / 3600000
        df.at[index, 'total_energy_MJ'] = total_energy_J / 1000000

    except Exception as e:
        print(f"Error processing batch at row {index + 2}: {str(e)}")
        continue


Processing batch: 100 L
Input values:
m_mix: 100
T_r: 30
T_0: 25
reaction_time: 172800
rate_heat_loss_perkelvin: 0.712
C_p: 4181.0
h_heat: 0.75

Heating energy calculation debug:
m_mix: 100
T_r_celsius: 30
T_0_celsius: 25
reaction_time: 172800
rate_heat_loss_perkelvin: 0.712
C_p: 4181.0
h_heat: 0.75
T_r (K): 303.15
T_0 (K): 298.15
Q_heat: 2090500.0
Q_loss: 615167.9999999999
Q_react: 3607557.3333333335

Filtration energy calculation debug:
dry_material_mass_kg: 1
use_upper_limit: False
dry_material_mass_ton: 0.001
Using lower limit: 1 kWh/ton
Calculated energy: 0.001 kWh

Drying energy calculation debug:
Cp_liq: 4184.0
m_liq: 2
T_boil_celsius: 100.0
T_0_celsius: 25
H_vap: 2260000.0
m_vap: 2
eta_dry: 0.8
T_boil (K): 373.15
T_0 (K): 298.15
Q_heat: 627600.0
Q_vap: 4520000.0
Q_dry: 6434500.0

Processing batch: 500 L
Input values:
m_mix: 500
T_r: 30
T_0: 25
reaction_time: 172800
rate_heat_loss_perkelvin: 2.081
C_p: 4181.0
h_heat: 0.75

Heating energy calculation debug:
m_mix: 500
T_r_celsiu

In [5]:
# Round the results
df['heating_energy_J'] = df['heating_energy_J'].round(3)
df['stirring_energy_J'] = df['stirring_energy_J'].round(3)
df['filtration_energy_kWh'] = df['filtration_energy_kWh'].round(3)
df['drying_energy_J'] = df['drying_energy_J'].round(3)
df['total_energy_kJ'] = df['total_energy_kJ'].round(3)
df['total_energy_kWh'] = df['total_energy_kWh'].round(3)
df['total_energy_MJ'] = df['total_energy_MJ'].round(3)

# Create mapping of new column names with descriptive headers
new_column_headers = {
    'heating_energy_J': 'Heating Energy\n(heating_energy_J)\n[J]',
    'stirring_energy_J': 'Stirring Energy\n(stirring_energy_J)\n[J]',
    'filtration_energy_kWh': 'Filtration Energy\n(filtration_energy_kWh)\n[kWh]',
    'drying_energy_J': 'Drying Energy\n(drying_energy_J)\n[J]',
    'total_energy_kJ': 'Total Energy\n(total_energy_kJ)\n[kJ]',
    'total_energy_kWh': 'Total Energy\n(total_energy_kWh)\n[kWh]',
    'total_energy_MJ': 'Total Energy\n(total_energy_MJ)\n[MJ]',
    'reaction_time': 'Reaction Time\n(reaction_time)\n[s]'
}

# Create mapping of original column names
original_columns = {}
for i, col in enumerate(original_column_names):
    if '\n' in str(col):
        var_name = col.split('\n')[1].strip('()')
        original_columns[var_name] = col

# Create output DataFrame
df_output = df.copy()

# Create new column names list
output_columns = []
for col in df_output.columns:
    if col in new_column_headers:
        output_columns.append(new_column_headers[col])
    elif col in original_columns:
        output_columns.append(original_columns[col])
    else:
        output_columns.append(col)

# Apply the new column names
df_output.columns = output_columns

# Display results
print("\nDetailed Energy Results:")
results_columns = [
    'batch_description',
    'heating_energy_J',
    'stirring_energy_J',
    'filtration_energy_kWh',
    'drying_energy_J',
    'total_energy_kJ',
    'total_energy_kWh',
    'total_energy_MJ'
]
print(df[results_columns])

# Save results with descriptive headers
output_filename = 'results_' + filename
df_output.to_excel(output_filename, index=False)
files.download(output_filename)

# Print column mapping for verification
print("\nColumn mapping verification:")
for old, new in zip(df.columns, df_output.columns):
    print(f"{old} -> {new}")


Detailed Energy Results:
  batch_description  heating_energy_J  stirring_energy_J  \
0             100 L      3.607557e+06       6.682092e+05   
1             500 L      1.633398e+07       1.960205e+06   
2            1000 L      3.167839e+07       3.115889e+06   
3            5000 L      1.504938e+08       9.101704e+06   
4           10000 L      2.963969e+08       1.442726e+07   

   filtration_energy_kWh  drying_energy_J  total_energy_kJ  total_energy_kWh  \
0                  0.001        6434500.0        10713.866             2.976   
1                  0.005       32172500.0        50484.684            14.024   
2                  0.010       64345000.0        99175.279            27.549   
3                  0.050      321725000.0       481500.539           133.750   
4                  0.100      643450000.0       954634.205           265.176   

   total_energy_MJ  
0           10.714  
1           50.485  
2           99.175  
3          481.501  
4          954.634  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Column mapping verification:
batch_description -> Batch Description
(batch_description)
[-]
date -> Process Date
(date)
[YYYY-MM-DD]
m_mix -> Mass of Reaction Mixture
(m_mix)
[kg]
T_r -> Reaction Temperature
(T_r)
[°C]
T_0 -> Starting Temperature
(T_0)
[°C]
reaction_time_hours -> Reaction Time
(reaction_time_hours)
[hours]
rate_heat_loss_perkelvin -> Heat Loss Rate per Kelvin
(rate_heat_loss_perkelvin)
[W/K]
C_p -> Specific Heat Capacity
(C_p)
[J/kg·K]
h_heat -> Heating Efficiency
(h_heat)
[-]
rho_mix -> Mixture Density
(rho_mix)
[kg/m³]
N -> Rotational Speed
(N)
[1/s]
d -> Impeller Diameter
(d)
[m]
Np -> Power Number
(Np)
[-]
h_stir -> Stirring Efficiency
(h_stir)
[-]
dry_material_mass_kg -> Mass of Dry Material
(dry_material_mass_kg)
[kg]
use_upper_limit_filtration -> Use Upper Energy Limit
(use_upper_limit_filtration)
[TRUE/FALSE]
m_liq -> Mass of Liquid
(m_liq)
[kg]
m_vap -> Mass of Vaporized Liquid
(m_vap)
[kg]
Cp_liq -> Liquid Specific Heat Capacity
(Cp_liq)
[J/kg·K]
T_boil -> B